## Sample Experiments

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/Users/shahules/Myprojects/synthetic-qa-paper/src')

In [3]:
from rag.config import ComponentConfig, Config,instantiate_component, DirectoryConfig, RetrieverConfig

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_prompt" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
directory = DirectoryConfig(name="../datasets/Sample_Docs_Markdown/")

In [5]:
config = Config(directory=directory)

In [6]:
config.model_dump()
config

Config(directory=DirectoryConfig(name='../datasets/Sample_Docs_Markdown/', params={'sample_size': 1, 'show_progress': True}), experiment_name='default_experiment', text_splitter=ComponentConfig(class_path='langchain_text_splitters.RecursiveCharacterTextSplitter', params={'chunk_size': 512, 'chunk_overlap': 100}), embedding_model=ComponentConfig(class_path='langchain_openai.OpenAIEmbeddings', params={}), llm=ComponentConfig(class_path='langchain_openai.ChatOpenAI', params={'model': 'gpt-4o'}), retriever=RetrieverConfig(type='vanilla', params={'search_kwargs': {'k': 2}}), model_prompt='Answer the user query based on the context')

In [7]:
from rag.rag import RAGBuilder

In [8]:
rag = RAGBuilder(config)

100%|███████████████████████████████████████████████| 1/1 [00:05<00:00,  5.93s/it]


In [9]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

In [10]:
sample = SingleTurnSample(user_input="What is an MoE?", reference="MoE is something")
dataset = EvaluationDataset(samples=[sample,sample])

In [11]:
dataset.to_pandas()

,user_input,reference
0,What is an MoE?,MoE is something
1,What is an MoE?,MoE is something


In [12]:
from ragas.metrics import ContextPrecision, FactualCorrectness
metrics = [FactualCorrectness()]

In [13]:
from ragas.llms import llm_factory
llm = llm_factory()

In [14]:
results = await rag.benchmark(dataset, metrics=metrics,eval_llm=llm)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
config.directory.model_dump_json()

'{"name":"../datasets/Sample_Docs_Markdown/","params":{"sample_size":1,"show_progress":true}}'

In [17]:
import os
os.path.join('ads','fss','fs')

'ads/fss/fs'